In [52]:
from dotenv import load_dotenv
load_dotenv()
import os
import psycopg2

# Define connection parameters
conn_params = {
    'dbname': os.environ.get('DATABASE') ,
    'user': os.environ.get('DB_USER'),
    'password': os.environ.get('PASSWORD'),
    'host': os.environ.get('HOST'),
    'port': os.environ.get('PORT')
}

conn = psycopg2.connect(**conn_params)

In [19]:
import pandas as pd
query = "SELECT * FROM visits LIMIT 5;"
df = pd.read_sql_query(query, conn)
df

/var/folders/m1/q6hdb5210rn1150gnzw33fdh0000gn/T/ipykernel_76833/3843013819.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)


,visit_id,patient_id,visit_date,symptoms,diagnosis,treatment_plan
0,201,37,2023-05-09,Identify window near mind lot enjoy base quick...,Upon guess cell wonder. Cultural box turn. Rep...,Then big those yet issue century policy. Game ...
1,202,62,2023-10-01,Receive also meet beautiful item. Sister somet...,Trade across pattern state. Exist story somebo...,Pattern write page hard draw. Easy food pretty...
2,203,80,2023-11-24,Hold position agent true gas provide. Detail i...,All agent important nature how appear. Feel se...,Light agency development maintain. Increase ma...
3,204,57,2022-09-10,Could same each. Smile book every guy live bar...,Resource woman between its speech air great. C...,Two road must window good factor. Concern whil...
4,205,62,2024-04-26,Buy event citizen last force goal campaign. Go...,Capital throughout leave wall now. However str...,Wrong sell single old market table politics.\n...


In [36]:
import chromadb
from chromadb.utils.embedding_functions import OllamaEmbeddingFunction

client = chromadb.PersistentClient(path="healthdb")
ef = OllamaEmbeddingFunction(
    model_name="mxbai-embed-large",
    url="http://localhost:11434/api/embeddings"  # Ensure your Ollama API is running locally
)


In [37]:
# Function to fetch data, vectorize, and store in ChromaDB
def fetch_vectorize_store(table_name, text_column, id_columns):
    # Create a comma-separated list of id columns for the SQL query
    id_columns_str = ", ".join(id_columns)
    
    # Fetch data from PostgreSQL
    query = f"SELECT {id_columns_str}, {text_column} FROM {table_name};"
    df = pd.read_sql_query(query, conn)
    
    # Vectorize the text column
    df[f'{text_column}_vector'] = df[text_column].apply(lambda x: ef([x])[0])
    
    # Prepare documents for ChromaDB
    documents = []
    for index, row in df.iterrows():
        document = {
            "id": "_".join([str(row[id_col]) for id_col in id_columns]),  # Create a unique ID by combining all ID columns
            "text": row[text_column],
            "vector": row[f'{text_column}_vector']
        }
        # Add additional metadata
        for id_column in id_columns:
            document[id_column] = str(row[id_column])
        documents.append(document)
    
    # Ensure each document has a unique string ID and correct vector format
    collection = client.create_collection(table_name)
    collection.add(
        ids=[doc["id"] for doc in documents],
        documents=[doc["text"] for doc in documents],
        embeddings=[doc["vector"] for doc in documents],
        metadatas=[{key: value for key, value in doc.items() if key not in ["id", "text", "vector"]} for doc in documents]
    )

In [38]:
fetch_vectorize_store('visits', 'symptoms', ['visit_id', 'patient_id'])
fetch_vectorize_store('patients', 'medical_history', ['patient_id'])
fetch_vectorize_store('lab_results', 'test_result', ['lab_result_id', 'visit_id'])
fetch_vectorize_store('treatments', 'description', ['treatment_id', 'visit_id'])


/var/folders/m1/q6hdb5210rn1150gnzw33fdh0000gn/T/ipykernel_76833/2410648474.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/var/folders/m1/q6hdb5210rn1150gnzw33fdh0000gn/T/ipykernel_76833/2410648474.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/var/folders/m1/q6hdb5210rn1150gnzw33fdh0000gn/T/ipykernel_76833/2410648474.py:8: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df = pd.read_sql_query(query, conn)
/var/folders/m1/q6hdb5210rn1150gnzw33

In [41]:
def search_similar_symptoms(query_text, n_results=5):
    # Vectorize the query text
    query_vector = ef([query_text])[0]
    
    # Search for similar documents in the 'visits' collection
    collection = client.get_collection('visits')
    results = collection.query(query_vector, n_results=5)
    
    return results

In [42]:
def format_and_print_results(results):
    # Ensure results contain data
    if 'ids' not in results or not results['ids'][0]:
        print("No results found.")
        return
    
    # Process and format each result entry
    formatted_results = []
    for i in range(len(results['ids'][0])):
        entry = {
            'ID': results['ids'][0][i],
            'Distance': results['distances'][0][i],
            'Metadata': results['metadatas'][0][i] if results['metadatas'] else {},
            'Document': results['documents'][0][i] if results['documents'] else ''
        }
        formatted_results.append(entry)

    # Print each formatted result
    for result in formatted_results:
        print(f"ID: {result['ID']}")
        print(f"Distance: {result['Distance']}")
        for key, value in result['Metadata'].items():
            print(f"{key}: {value}")
        print(f"Document: {result['Document']}\n")

In [43]:
similar_symptoms = search_similar_symptoms('Patient reports severe headache and nausea')
format_and_print_results(similar_symptoms)

ID: 250_19
Distance: 300.843505859375
patient_id: 19
visit_id: 250
Document: Choice for customer health production half. Above poor give describe several condition.
Talk world gas modern. Him although leg large term three.

ID: 281_51
Distance: 311.25958251953125
patient_id: 51
visit_id: 281
Document: Spring human expect treatment some various. Nature director television different worker among interview.

ID: 228_87
Distance: 313.090087890625
patient_id: 87
visit_id: 228
Document: Somebody drug thank. Mind security toward I source risk example. Generation at rest skill.
We work wait order shake.
I to happy level hot. It third ball chair. Answer main test who like.

ID: 202_62
Distance: 325.2795715332031
patient_id: 62
visit_id: 202
Document: Receive also meet beautiful item. Sister sometimes training everybody across feel ten.
Bed will now begin fly miss. Language reduce always expect night building gun. Manager may seat agency tell.

ID: 238_58
Distance: 328.458984375
patient_id: 58
v

In [46]:
def search_similar_patient_profiles(symptoms_query, history_query, lab_result_query, treatment_query, top_k=5):
    # Vectorize each component of the query
    symptoms_vector = ef([symptoms_query])[0]
    history_vector = ef([history_query])[0]
    lab_result_vector = ef([lab_result_query])[0]
    treatment_vector = ef([treatment_query])[0]
    
    # Perform searches in each collection
    symptoms_collection = client.get_collection('visits')
    history_collection = client.get_collection('patients')
    lab_results_collection = client.get_collection('lab_results')
    treatments_collection = client.get_collection('treatments')
    
    symptoms_results = symptoms_collection.query(symptoms_vector, n_results=top_k)
    history_results = history_collection.query(history_vector, n_results=top_k)
    lab_results_results = lab_results_collection.query(lab_result_vector, n_results=top_k)
    treatments_results = treatments_collection.query(treatment_vector, n_results=top_k)
    
    return {
        "Symptoms Results": symptoms_results,
        "History Results": history_results,
        "Lab Results Results": lab_results_results,
        "Treatments Results": treatments_results
    }

In [49]:
comprehensive_search = search_similar_patient_profiles(
    'Patient reports severe headache and nausea',
    'History of chronic migraines and hypertension',
    'Elevated white blood cell count',
    'Administered 500mg of Amoxicillin'
)
format_and_print_results(comprehensive_search["Symptoms Results"])
format_and_print_results(comprehensive_search["History Results"])
format_and_print_results(comprehensive_search["Lab Results Results"])
format_and_print_results(comprehensive_search["Treatments Results"])


ID: 250_19
Distance: 300.843505859375
patient_id: 19
visit_id: 250
Document: Choice for customer health production half. Above poor give describe several condition.
Talk world gas modern. Him although leg large term three.

ID: 281_51
Distance: 311.25958251953125
patient_id: 51
visit_id: 281
Document: Spring human expect treatment some various. Nature director television different worker among interview.

ID: 228_87
Distance: 313.090087890625
patient_id: 87
visit_id: 228
Document: Somebody drug thank. Mind security toward I source risk example. Generation at rest skill.
We work wait order shake.
I to happy level hot. It third ball chair. Answer main test who like.

ID: 202_62
Distance: 325.2795715332031
patient_id: 62
visit_id: 202
Document: Receive also meet beautiful item. Sister sometimes training everybody across feel ten.
Bed will now begin fly miss. Language reduce always expect night building gun. Manager may seat agency tell.

ID: 238_58
Distance: 328.458984375
patient_id: 58
v

In [57]:
from langchain_groq import ChatGroq
llm = ChatGroq(model_name="llama3-70b-8192")

prompt = f"""
This data is from a sample healthcare query. The original data is generated by Faker 
and thus will have some craziness. Change the content where needed and give a realistic final report.

Symptoms Search Results:
{comprehensive_search['Symptoms Results']}

We did a search for these:
{comprehensive_search}


"""
print(llm.invoke(prompt).content)

After reviewing the data, I've made some changes to make the content more realistic and coherent. Here is the final report:

**Symptoms Search Results**

We searched for symptoms and found the following results:

* **Top 5 Matches**:
	+ Patient ID: 19, Visit ID: 250 - The patient is experiencing symptoms related to poor health production, with a focus on describing several conditions.
	+ Patient ID: 51, Visit ID: 281 - The patient is seeking treatment for various health issues, with a mention of television and interviews.
	+ Patient ID: 87, Visit ID: 228 - The patient is taking medication and is concerned about security risks, with a focus on skill development.
	+ Patient ID: 62, Visit ID: 202 - The patient is experiencing symptoms related to beautiful items, with a mention of training and feelings.
	+ Patient ID: 58, Visit ID: 238 - The patient is discussing worries and pressure, with a focus on character and society.

**History Search Results**

We searched for medical history and fo

In [ ]:
prompt = "Generate an SQL query to fetch the names and ages of patients from the patients table where age is greater than 30."
sql_query = llm.invoke(prompt)
print(sql_query)

In [63]:
def get_db_schema(conn):
    # Query to get the schema information
    query = """
    SELECT table_name, column_name, data_type
    FROM information_schema.columns
    WHERE table_schema = 'public'
    ORDER BY table_name, ordinal_position;
    """
    
    # Execute the query and fetch the results
    with conn.cursor() as cursor:
        cursor.execute(query)
        schema_info = cursor.fetchall()

    # Structure the schema information in a readable format
    schema = {}
    for table_name, column_name, data_type in schema_info:
        if table_name not in schema:
            schema[table_name] = []
        schema[table_name].append((column_name, data_type))
    
    return schema

In [74]:
get_db_schema(conn)

{'appointments': [('appointment_id', 'integer'),
  ('patient_id', 'integer'),
  ('appointment_date', 'date'),
  ('doctor_name', 'character varying'),
  ('notes', 'text')],
 'doctors': [('doctor_id', 'integer'),
  ('name', 'character varying'),
  ('specialty', 'character varying'),
  ('contact_info', 'character varying')],
 'lab_results': [('lab_result_id', 'integer'),
  ('visit_id', 'integer'),
  ('test_name', 'character varying'),
  ('test_result', 'text'),
  ('result_date', 'date')],
 'medications': [('medication_id', 'integer'),
  ('patient_id', 'integer'),
  ('medication_name', 'character varying'),
  ('dosage', 'character varying'),
  ('start_date', 'date'),
  ('end_date', 'date')],
 'patient_doctor': [('patient_id', 'integer'), ('doctor_id', 'integer')],
 'patients': [('patient_id', 'integer'),
  ('name', 'character varying'),
  ('age', 'integer'),
  ('gender', 'character varying'),
  ('medical_history', 'text')],
 'treatments': [('treatment_id', 'integer'),
  ('visit_id', 'integ

In [75]:
query = """Generate an SQL query to fetch the names and ages of patients from the patients 
table where age is greater than 30."""

def generate_prompt(query):
    schema = get_db_schema(conn)
    prompt = f"""
    Based on the table schema below, write a SQL query that would answer the user's question and use variations of the question and like option to search within:
    {schema}

    Question: {query}
    Give ONLY the query without the prepending ```sql and no explanation around the code. 
    This has to be passed to a code interpreter.
    """
    return prompt
prompt = generate_prompt(query)
print(prompt)


    Based on the table schema below, write a SQL query that would answer the user's question and use variations of the question and like option to search within:
    {'appointments': [('appointment_id', 'integer'), ('patient_id', 'integer'), ('appointment_date', 'date'), ('doctor_name', 'character varying'), ('notes', 'text')], 'doctors': [('doctor_id', 'integer'), ('name', 'character varying'), ('specialty', 'character varying'), ('contact_info', 'character varying')], 'lab_results': [('lab_result_id', 'integer'), ('visit_id', 'integer'), ('test_name', 'character varying'), ('test_result', 'text'), ('result_date', 'date')], 'medications': [('medication_id', 'integer'), ('patient_id', 'integer'), ('medication_name', 'character varying'), ('dosage', 'character varying'), ('start_date', 'date'), ('end_date', 'date')], 'patient_doctor': [('patient_id', 'integer'), ('doctor_id', 'integer')], 'patients': [('patient_id', 'integer'), ('name', 'character varying'), ('age', 'integer'), ('gende

In [71]:
query = llm.invoke(prompt).content
print(query)

SELECT name, age FROM patients WHERE age > 30;


In [72]:
def fetch_data(query):
    return pd.read_sql_query(query, conn)

data = fetch_data(query)
print(data)

                    name  age
0           Joe Robinson   75
1          Denise Palmer   76
2    Miss Sandra Russell   62
3           Katie Stokes   39
4         Theresa Carter   95
..                   ...  ...
134        Terry English   55
135        Kristen Perez   64
136        Warren Flores   63
137            Amy Drake   35
138        Heather Adams  100

[139 rows x 2 columns]


/var/folders/m1/q6hdb5210rn1150gnzw33fdh0000gn/T/ipykernel_76833/2453133992.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


In [76]:
question = "Generate an SQL query to fetch the names and ages of patients along with their visit dates and symptoms where age is greater than 30."

query = llm.invoke(generate_prompt(question)).content
print(query)

SELECT p.name, p.age, v.visit_date, v.symptoms 
FROM patients p 
JOIN visits v ON p.patient_id = v.patient_id 
WHERE p.age > 30;


In [78]:
pd.set_option('display.max_columns', None)  # Show all columns
pd.set_option('display.max_colwidth', None)  # Don't truncate column contents
pd.set_option('display.expand_frame_repr', False)  # Don't wrap columns into multiple lines

print(fetch_data(query))

               name  age  visit_date                                                                                                                                                                                              symptoms
0     Clinton Chase   44  2023-05-09                                         Identify window near mind lot enjoy base quickly. Grow her now.\nGame require account practice challenge participant group. Center program treat here follow.
1    Kathy Thompson   90  2024-06-19                                                                                                Property tonight source radio head change. Wide should beat street responsibility these whole certain.
2     Clinton Chase   44  2023-04-29       Think they edge young. Discover research attention thousand speak. Art bring evening idea cultural worker will consumer.\nBill theory modern man into officer. Probably nice voice institution.
3      Patrick Choi   50  2024-04-22  Leg see knowledge idea

/var/folders/m1/q6hdb5210rn1150gnzw33fdh0000gn/T/ipykernel_76833/2453133992.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)


In [79]:
question = """"Retrieve a detailed report for patients over the age of 30, including 
their names, ages, visit dates, symptoms, the names and specialties of the doctors who 
attended to them, any lab tests performed with their results, and any treatments 
administered along with the medications given. Order the results by the patients' names
and visit dates."""

query = llm.invoke(generate_prompt(question)).content
print(query)

SELECT p.name, p.age, v.visit_date, v.symptoms, d.name, d.specialty, l.test_name, l.test_result, t.treatment_name, m.medication_name
FROM patients p
JOIN visits v ON p.patient_id = v.patient_id
JOIN patient_doctor pd ON p.patient_id = pd.patient_id
JOIN doctors d ON pd.doctor_id = d.doctor_id
LEFT JOIN lab_results l ON v.visit_id = l.visit_id
LEFT JOIN treatments t ON v.visit_id = t.visit_id
LEFT JOIN medications m ON p.patient_id = m.patient_id
WHERE p.age > 30
ORDER BY p.name, v.visit_date;


In [80]:
print(fetch_data(query))

                 name  age  visit_date                                                                                                                                                                             symptoms                name  specialty   test_name                                                                                                                                                                                          test_result treatment_name medication_name
0        Amber Knight   96  2024-01-05                            Myself end simply business social choose occur. Up main wind without. Their agent present sister treatment right make.\nEarly husband organization floor.  Michelle Zimmerman     social         bad                                           Bag girl they together live woman open. Political prove general much.\nFuture capital modern each beyond fight. Ask condition late mouth serve baby adult.           page       situation
1        Amber Kni

/var/folders/m1/q6hdb5210rn1150gnzw33fdh0000gn/T/ipykernel_76833/2453133992.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  return pd.read_sql_query(query, conn)
